## [1.2.1] Steady-state assumption-based cross-iteration approximation for key radicals
### Here we approximate the concentrations of the 3 critical radicals: O[¹D], HO₂, and CH₃O₂
###### Updated on 27 April 2025
#### We use cross-iteration approximation for HO₂ and CH₃O₂
#### O[¹D] can be approximated directly

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('/Users/csuen/Desktop/Sun-NG-24/radicals/UKESM1-0-LL_2010_2014(ver231206).csv')

We need to calculate the temperature-dependent kinetic coefficients

In [3]:
dataset['k1'] = 2.15*10**(-11)*np.exp(110/dataset['tas'])
dataset['k2'] = 3.30*10**(-11)*np.exp(55/dataset['tas'])
dataset['k3'] = 1.63*10**(-10)*np.exp(60/dataset['tas'])
dataset['k4'] = 4.30*10**(-32)*(dataset['tas']/300)**(-1.2)*dataset['n2_mole']
dataset['k5'] = 1.44*10**(-13)

dataset['k6'] = 5.40*10**(-12)*np.exp(135/dataset['tas'])
dataset['k7'] = 1.70*10**(-12)*np.exp(-940/dataset['tas'])
dataset['k8'] = 2.45*10**(-12)*np.exp(-600/dataset['tas'])
dataset['k9'] = 4.01*10**(-12)*np.exp(190/dataset['tas'])
dataset['k9a'] = 2.12*10**(-12)*np.exp(190/dataset['tas'])
dataset['k9b'] = 1.89*10**(-12)*np.exp(190/dataset['tas'])

dataset['k10'] = 3.30*10**(-12)*np.exp(270/dataset['tas'])
dataset['k11'] = 2.03*10**(-16)*(dataset['tas']/300)**4.57*np.exp(693/dataset['tas'])
dataset['k12'] = 3.00*10**(-13)*np.exp(460/dataset['tas'])
dataset['k13'] = 4.80*10**(-11)*np.exp(250/dataset['tas'])

dataset['k14'] = 2.30*10**(-12)*np.exp(360/dataset['tas'])
dataset['k15'] = 3.80*10**(-13)*np.exp(780/dataset['tas'])
dataset['k15a'] = dataset['k15']/(1+498*np.exp(-1160/dataset['tas']))
dataset['k15b'] = dataset['k15']-dataset['k15a']
dataset['k16'] = 1.03*10**(-13)*np.exp(365/dataset['tas'])

#### Converse mass fraction of H$_2$O into standard unit: molecule cm$^{-3}$

In [4]:
dataset['n2_mole'] = dataset['ps']/8.314/dataset['tas']*6.022*10**23*10**(-6)*0.7808
dataset['o2_mole'] = dataset['ps']/8.314/dataset['tas']*6.022*10**23*10**(-6)*0.2095
dataset['h2o_mole'] = dataset['h2o']/18*6.022*10**23/(8.314*dataset['tas']/dataset['ps']/28.96*10**6)
dataset['o3_mole']   = dataset['o3']  *1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['no2_mole']  = dataset['no2'] *1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['no_mole']   = dataset['no']  *1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['oh_mole']   = dataset['oh']  *1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['co_mole']   = dataset['co']  *1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['hcho_mole'] = dataset['hcho']*1.01325*10**5/dataset['tas']*7.243*10**7*10**9
dataset['ch4_mole']  = dataset['ch4'] *1.01325*10**5/dataset['tas']*7.243*10**7*10**9

#### Calculate the radical concentration of [O(${^1}$D)] via steady-state approximation

In [5]:
dataset['o1d_mole'] = dataset['photo1d']*dataset['o3_mole']/(dataset['k1']*dataset['n2_mole'] + dataset['k2']*dataset['o2_mole'] + dataset['k3']*dataset['h2o_mole'])

#### Set initial level of HO${_2}$ radical for iteration 

In [6]:
dataset['lossco_mole'] = dataset['lossco']*6.022*10**23*10**(-6)

dataset['A_0'] = 2*dataset['k12']
dataset['B_0'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole']
dataset['C_0'] = -(dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole'])

dataset['ho2_0'] = (-dataset['B_0']+(dataset['B_0']*dataset['B_0']-4*dataset['A_0']*dataset['C_0'])**(0.5))/(2*dataset['A_0'])

#### Set initial level of CH${_3}$O${_2}$ radical for iteration 

In [7]:
dataset['lossch4_mole'] = dataset['lossch4']*6.022*10**23*10**(-6)

dataset['D_0'] = 2*dataset['k16']
dataset['E_0'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_0']
dataset['F_0'] = -(dataset['lossch4_mole'])

dataset['ch3o2_0'] = (-dataset['E_0']+(dataset['E_0']*dataset['E_0']-4*dataset['D_0']*dataset['F_0'])**(0.5))/(2*dataset['D_0'])

#### Set initial level of CH${_3}$O${_2}$H radical for iteration 

In [8]:
dataset['ch3ooh_0'] = dataset['k15b']*dataset['ho2_0']*dataset['ch3o2_0']/(dataset['k9']*dataset['oh_mole'])

### Now start iteration

In [9]:
dataset['A_1'] = 2*dataset['k12']
dataset['B_1'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_0']
dataset['C_1'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_0'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_1'] = (-dataset['B_1']+(dataset['B_1']*dataset['B_1']-4*dataset['A_1']*dataset['C_1'])**(0.5))/(2*dataset['A_1'])

dataset['D_1'] = 2*dataset['k16']
dataset['E_1'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_1']
dataset['F_1'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_0']*dataset['oh_mole']) 

dataset['ch3o2_1'] = (-dataset['E_1']+(dataset['E_1']*dataset['E_1']-4*dataset['D_1']*dataset['F_1'])**(0.5))/(2*dataset['D_1'])

dataset['ch3ooh_1'] = dataset['k15b']*dataset['ho2_1']*dataset['ch3o2_1']/(dataset['k9']*dataset['oh_mole'])

In [10]:
dataset['A_2'] = 2*dataset['k12']
dataset['B_2'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_1']
dataset['C_2'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_1'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_2'] = (-dataset['B_2']+(dataset['B_2']*dataset['B_2']-4*dataset['A_2']*dataset['C_2'])**(0.5))/(2*dataset['A_2'])

dataset['D_2'] = 2*dataset['k16']
dataset['E_2'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_2']
dataset['F_2'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_1']*dataset['oh_mole']) 

dataset['ch3o2_2'] = (-dataset['E_2']+(dataset['E_2']*dataset['E_2']-4*dataset['D_2']*dataset['F_2'])**(0.5))/(2*dataset['D_2'])

dataset['ch3ooh_2'] = dataset['k15b']*dataset['ho2_2']*dataset['ch3o2_2']/(dataset['k9']*dataset['oh_mole'])

In [11]:
dataset['A_3'] = 2*dataset['k12']
dataset['B_3'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_2']
dataset['C_3'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_2'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_3'] = (-dataset['B_3']+(dataset['B_3']*dataset['B_3']-4*dataset['A_3']*dataset['C_3'])**(0.5))/(2*dataset['A_3'])

dataset['D_3'] = 2*dataset['k16']
dataset['E_3'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_3']
dataset['F_3'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_2']*dataset['oh_mole']) 

dataset['ch3o2_3'] = (-dataset['E_3']+(dataset['E_3']*dataset['E_3']-4*dataset['D_3']*dataset['F_3'])**(0.5))/(2*dataset['D_3'])

dataset['ch3ooh_3'] = dataset['k15b']*dataset['ho2_3']*dataset['ch3o2_3']/(dataset['k9']*dataset['oh_mole'])

In [12]:
dataset['A_4'] = 2*dataset['k12']
dataset['B_4'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_3']
dataset['C_4'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_3'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_4'] = (-dataset['B_4']+(dataset['B_4']*dataset['B_4']-4*dataset['A_4']*dataset['C_4'])**(0.5))/(2*dataset['A_4'])

dataset['D_4'] = 2*dataset['k16']
dataset['E_4'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_4']
dataset['F_4'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_3']*dataset['oh_mole']) 

dataset['ch3o2_4'] = (-dataset['E_4']+(dataset['E_4']*dataset['E_4']-4*dataset['D_4']*dataset['F_4'])**(0.5))/(2*dataset['D_4'])

dataset['ch3ooh_4'] = dataset['k15b']*dataset['ho2_4']*dataset['ch3o2_4']/(dataset['k9']*dataset['oh_mole'])

In [13]:
dataset['A_5'] = 2*dataset['k12']
dataset['B_5'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_4']
dataset['C_5'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_4'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_5'] = (-dataset['B_5']+(dataset['B_5']*dataset['B_5']-4*dataset['A_5']*dataset['C_5'])**(0.5))/(2*dataset['A_5'])

dataset['D_5'] = 2*dataset['k16']
dataset['E_5'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_5']
dataset['F_5'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_4']*dataset['oh_mole']) 

dataset['ch3o2_5'] = (-dataset['E_5']+(dataset['E_5']*dataset['E_5']-4*dataset['D_5']*dataset['F_5'])**(0.5))/(2*dataset['D_5'])

dataset['ch3ooh_5'] = dataset['k15b']*dataset['ho2_5']*dataset['ch3o2_5']/(dataset['k9']*dataset['oh_mole'])

In [14]:
dataset['A_6'] = 2*dataset['k12']
dataset['B_6'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_5']
dataset['C_6'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_5'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_6'] = (-dataset['B_6']+(dataset['B_6']*dataset['B_6']-4*dataset['A_6']*dataset['C_6'])**(0.5))/(2*dataset['A_6'])

dataset['D_6'] = 2*dataset['k16']
dataset['E_6'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_6']
dataset['F_6'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_5']*dataset['oh_mole']) 

dataset['ch3o2_6'] = (-dataset['E_6']+(dataset['E_6']*dataset['E_6']-4*dataset['D_6']*dataset['F_6'])**(0.5))/(2*dataset['D_6'])

dataset['ch3ooh_6'] = dataset['k15b']*dataset['ho2_6']*dataset['ch3o2_6']/(dataset['k9']*dataset['oh_mole'])

In [15]:
dataset['A_7'] = 2*dataset['k12']
dataset['B_7'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_6']
dataset['C_7'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_6'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_7'] = (-dataset['B_7']+(dataset['B_7']*dataset['B_7']-4*dataset['A_7']*dataset['C_7'])**(0.5))/(2*dataset['A_7'])

dataset['D_7'] = 2*dataset['k16']
dataset['E_7'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_7']
dataset['F_7'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_6']*dataset['oh_mole']) 

dataset['ch3o2_7'] = (-dataset['E_7']+(dataset['E_7']*dataset['E_7']-4*dataset['D_7']*dataset['F_7'])**(0.5))/(2*dataset['D_7'])

dataset['ch3ooh_7'] = dataset['k15b']*dataset['ho2_7']*dataset['ch3o2_7']/(dataset['k9']*dataset['oh_mole'])

In [16]:
dataset['A_8'] = 2*dataset['k12']
dataset['B_8'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_7']
dataset['C_8'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_7'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_8'] = (-dataset['B_8']+(dataset['B_8']*dataset['B_8']-4*dataset['A_8']*dataset['C_8'])**(0.5))/(2*dataset['A_8'])

dataset['D_8'] = 2*dataset['k16']
dataset['E_8'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_8']
dataset['F_8'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_7']*dataset['oh_mole']) 

dataset['ch3o2_8'] = (-dataset['E_8']+(dataset['E_8']*dataset['E_8']-4*dataset['D_8']*dataset['F_8'])**(0.5))/(2*dataset['D_8'])

dataset['ch3ooh_8'] = dataset['k15b']*dataset['ho2_8']*dataset['ch3o2_8']/(dataset['k9']*dataset['oh_mole'])

In [17]:
dataset['A_9'] = 2*dataset['k12']
dataset['B_9'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_8']
dataset['C_9'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_8'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_9'] = (-dataset['B_9']+(dataset['B_9']*dataset['B_9']-4*dataset['A_9']*dataset['C_9'])**(0.5))/(2*dataset['A_9'])

dataset['D_9'] = 2*dataset['k16']
dataset['E_9'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_9']
dataset['F_9'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_8']*dataset['oh_mole']) 

dataset['ch3o2_9'] = (-dataset['E_9']+(dataset['E_9']*dataset['E_9']-4*dataset['D_9']*dataset['F_9'])**(0.5))/(2*dataset['D_9'])

dataset['ch3ooh_9'] = dataset['k15b']*dataset['ho2_9']*dataset['ch3o2_9']/(dataset['k9']*dataset['oh_mole'])

In [18]:
dataset['A_10'] = 2*dataset['k12']
dataset['B_10'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_9']
dataset['C_10'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_9'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_10'] = (-dataset['B_10']+(dataset['B_10']*dataset['B_10']-4*dataset['A_10']*dataset['C_10'])**(0.5))/(2*dataset['A_10'])

dataset['D_10'] = 2*dataset['k16']
dataset['E_10'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_10']
dataset['F_10'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_9']*dataset['oh_mole']) 

dataset['ch3o2_10'] = (-dataset['E_10']+(dataset['E_10']*dataset['E_10']-4*dataset['D_10']*dataset['F_10'])**(0.5))/(2*dataset['D_10'])

dataset['ch3ooh_10'] = dataset['k15b']*dataset['ho2_10']*dataset['ch3o2_10']/(dataset['k9']*dataset['oh_mole'])

In [19]:
dataset['A_11'] = 2*dataset['k12']
dataset['B_11'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_10']
dataset['C_11'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_10'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_11'] = (-dataset['B_11']+(dataset['B_11']*dataset['B_11']-4*dataset['A_11']*dataset['C_11'])**(0.5))/(2*dataset['A_11'])

dataset['D_11'] = 2*dataset['k16']
dataset['E_11'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_11']
dataset['F_11'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_10']*dataset['oh_mole']) 

dataset['ch3o2_11'] = (-dataset['E_11']+(dataset['E_11']*dataset['E_11']-4*dataset['D_11']*dataset['F_11'])**(0.5))/(2*dataset['D_11'])

dataset['ch3ooh_11'] = dataset['k15b']*dataset['ho2_11']*dataset['ch3o2_11']/(dataset['k9']*dataset['oh_mole'])

In [20]:
dataset['A_12'] = 2*dataset['k12']
dataset['B_12'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_11']
dataset['C_12'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_11'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_12'] = (-dataset['B_12']+(dataset['B_12']*dataset['B_12']-4*dataset['A_12']*dataset['C_12'])**(0.5))/(2*dataset['A_12'])

dataset['D_12'] = 2*dataset['k16']
dataset['E_12'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_12']
dataset['F_12'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_11']*dataset['oh_mole']) 

dataset['ch3o2_12'] = (-dataset['E_12']+(dataset['E_12']*dataset['E_12']-4*dataset['D_12']*dataset['F_12'])**(0.5))/(2*dataset['D_12'])

dataset['ch3ooh_12'] = dataset['k15b']*dataset['ho2_12']*dataset['ch3o2_12']/(dataset['k9']*dataset['oh_mole'])

In [21]:
dataset['A_13'] = 2*dataset['k12']
dataset['B_13'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_12']
dataset['C_13'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_12'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_13'] = (-dataset['B_13']+(dataset['B_13']*dataset['B_13']-4*dataset['A_13']*dataset['C_13'])**(0.5))/(2*dataset['A_13'])

dataset['D_13'] = 2*dataset['k16']
dataset['E_13'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_13']
dataset['F_13'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_12']*dataset['oh_mole']) 

dataset['ch3o2_13'] = (-dataset['E_13']+(dataset['E_13']*dataset['E_13']-4*dataset['D_13']*dataset['F_13'])**(0.5))/(2*dataset['D_13'])

dataset['ch3ooh_13'] = dataset['k15b']*dataset['ho2_13']*dataset['ch3o2_13']/(dataset['k9']*dataset['oh_mole'])

In [22]:
dataset['A_14'] = 2*dataset['k12']
dataset['B_14'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_13']
dataset['C_14'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_13'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_14'] = (-dataset['B_14']+(dataset['B_14']*dataset['B_14']-4*dataset['A_14']*dataset['C_14'])**(0.5))/(2*dataset['A_14'])

dataset['D_14'] = 2*dataset['k16']
dataset['E_14'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_14']
dataset['F_14'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_13']*dataset['oh_mole']) 

dataset['ch3o2_14'] = (-dataset['E_14']+(dataset['E_14']*dataset['E_14']-4*dataset['D_14']*dataset['F_14'])**(0.5))/(2*dataset['D_14'])

dataset['ch3ooh_14'] = dataset['k15b']*dataset['ho2_14']*dataset['ch3o2_14']/(dataset['k9']*dataset['oh_mole'])

In [23]:
dataset['A_15'] = 2*dataset['k12']
dataset['B_15'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_14']
dataset['C_15'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_14'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_15'] = (-dataset['B_15']+(dataset['B_15']*dataset['B_15']-4*dataset['A_15']*dataset['C_15'])**(0.5))/(2*dataset['A_15'])

dataset['D_15'] = 2*dataset['k16']
dataset['E_15'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_15']
dataset['F_15'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_14']*dataset['oh_mole']) 

dataset['ch3o2_15'] = (-dataset['E_15']+(dataset['E_15']*dataset['E_15']-4*dataset['D_15']*dataset['F_15'])**(0.5))/(2*dataset['D_15'])

dataset['ch3ooh_15'] = dataset['k15b']*dataset['ho2_15']*dataset['ch3o2_15']/(dataset['k9']*dataset['oh_mole'])

In [24]:
dataset['A_16'] = 2*dataset['k12']
dataset['B_16'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_15']
dataset['C_16'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_15'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_16'] = (-dataset['B_16']+(dataset['B_16']*dataset['B_16']-4*dataset['A_16']*dataset['C_16'])**(0.5))/(2*dataset['A_16'])

dataset['D_16'] = 2*dataset['k16']
dataset['E_16'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_16']
dataset['F_16'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_15']*dataset['oh_mole']) 

dataset['ch3o2_16'] = (-dataset['E_16']+(dataset['E_16']*dataset['E_16']-4*dataset['D_16']*dataset['F_16'])**(0.5))/(2*dataset['D_16'])

dataset['ch3ooh_16'] = dataset['k15b']*dataset['ho2_16']*dataset['ch3o2_16']/(dataset['k9']*dataset['oh_mole'])

In [25]:
dataset['A_17'] = 2*dataset['k12']
dataset['B_17'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_16']
dataset['C_17'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_16'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_17'] = (-dataset['B_17']+(dataset['B_17']*dataset['B_17']-4*dataset['A_17']*dataset['C_17'])**(0.5))/(2*dataset['A_17'])

dataset['D_17'] = 2*dataset['k16']
dataset['E_17'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_17']
dataset['F_17'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_16']*dataset['oh_mole']) 

dataset['ch3o2_17'] = (-dataset['E_17']+(dataset['E_17']*dataset['E_17']-4*dataset['D_17']*dataset['F_17'])**(0.5))/(2*dataset['D_17'])

dataset['ch3ooh_17'] = dataset['k15b']*dataset['ho2_17']*dataset['ch3o2_17']/(dataset['k9']*dataset['oh_mole'])

In [26]:
dataset['A_18'] = 2*dataset['k12']
dataset['B_18'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_17']
dataset['C_18'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_17'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_18'] = (-dataset['B_18']+(dataset['B_18']*dataset['B_18']-4*dataset['A_18']*dataset['C_18'])**(0.5))/(2*dataset['A_18'])

dataset['D_18'] = 2*dataset['k16']
dataset['E_18'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_18']
dataset['F_18'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_17']*dataset['oh_mole']) 

dataset['ch3o2_18'] = (-dataset['E_18']+(dataset['E_18']*dataset['E_18']-4*dataset['D_18']*dataset['F_18'])**(0.5))/(2*dataset['D_18'])

dataset['ch3ooh_18'] = dataset['k15b']*dataset['ho2_18']*dataset['ch3o2_18']/(dataset['k9']*dataset['oh_mole'])

In [27]:
dataset['A_19'] = 2*dataset['k12']
dataset['B_19'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_18']
dataset['C_19'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_18'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_19'] = (-dataset['B_19']+(dataset['B_19']*dataset['B_19']-4*dataset['A_19']*dataset['C_19'])**(0.5))/(2*dataset['A_19'])

dataset['D_19'] = 2*dataset['k16']
dataset['E_19'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_19']
dataset['F_19'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_18']*dataset['oh_mole']) 

dataset['ch3o2_19'] = (-dataset['E_19']+(dataset['E_19']*dataset['E_19']-4*dataset['D_19']*dataset['F_19'])**(0.5))/(2*dataset['D_19'])

dataset['ch3ooh_19'] = dataset['k15b']*dataset['ho2_19']*dataset['ch3o2_19']/(dataset['k9']*dataset['oh_mole'])

In [28]:
dataset['A_20'] = 2*dataset['k12']
dataset['B_20'] = dataset['k10']*dataset['no_mole'] + dataset['k11']*dataset['o3_mole'] + dataset['k13']*dataset['oh_mole'] + dataset['k15']*dataset['ch3o2_19']
dataset['C_20'] = -(dataset['k14']*dataset['no_mole']*dataset['ch3o2_19'] + dataset['k6']*dataset['oh_mole']*dataset['hcho_mole'] + dataset['k7']*dataset['oh_mole']*dataset['o3_mole'] + dataset['lossco_mole']) 

dataset['ho2_20'] = (-dataset['B_20']+(dataset['B_20']*dataset['B_20']-4*dataset['A_20']*dataset['C_20'])**(0.5))/(2*dataset['A_20'])

dataset['D_20'] = 2*dataset['k16']
dataset['E_20'] = dataset['k14']*dataset['no_mole'] + dataset['k15']*dataset['ho2_20']
dataset['F_20'] = -(dataset['lossch4_mole'] + dataset['k9a']*dataset['ch3ooh_19']*dataset['oh_mole']) 

dataset['ch3o2_20'] = (-dataset['E_20']+(dataset['E_20']*dataset['E_20']-4*dataset['D_20']*dataset['F_20'])**(0.5))/(2*dataset['D_20'])

dataset['ch3ooh_20'] = dataset['k15b']*dataset['ho2_20']*dataset['ch3o2_20']/(dataset['k9']*dataset['oh_mole'])

#### Results summary for global and residential land surface

In [29]:
ho2_mean = dataset[['ho2_0','ho2_1','ho2_2','ho2_3','ho2_4','ho2_5',
                    'ho2_6','ho2_7','ho2_8','ho2_9','ho2_10',
                    'ho2_11','ho2_12','ho2_13','ho2_14','ho2_15',
                    'ho2_16','ho2_17','ho2_18','ho2_19','ho2_20']].mean()

In [30]:
ch3o2_mean = dataset[['ch3o2_0','ch3o2_1','ch3o2_2','ch3o2_3','ch3o2_4','ch3o2_5',
                      'ch3o2_6','ch3o2_7','ch3o2_8','ch3o2_9','ch3o2_10',
                      'ch3o2_11','ch3o2_12','ch3o2_13','ch3o2_14','ch3o2_15',
                      'ch3o2_16','ch3o2_17','ch3o2_18','ch3o2_19','ch3o2_20']].mean()

In [31]:
ch3ooh_mean = dataset[['ch3ooh_0','ch3ooh_1','ch3ooh_2','ch3ooh_3','ch3ooh_4','ch3ooh_5',
                       'ch3ooh_6','ch3ooh_7','ch3ooh_8','ch3ooh_9','ch3ooh_10',
                       'ch3ooh_11','ch3ooh_12','ch3ooh_13','ch3ooh_14','ch3ooh_15',
                       'ch3ooh_16','ch3ooh_17','ch3ooh_18','ch3ooh_19','ch3ooh_20']].mean()

In [32]:
global_seq = pd.DataFrame()
global_seq['iteration'] = np.arange(0,21,1)
global_seq['ho2_mean_glob'] = ho2_mean.values
global_seq['ch3o2_mean_glob'] = ch3o2_mean.values
global_seq['ch3ooh_mean_glob'] = ch3ooh_mean.values

In [33]:
land = pd.read_csv('/Users/csuen/Desktop/Sun-NG-24/global_grid_2x2_label_continent.csv')
dataset_land = dataset.merge(land, how = 'left', left_on=['longitude','latitude'], right_on=['longitude','latitude'])
dataset_land = dataset_land[dataset_land['land_global']>0]
dataset_land

,longitude,latitude,t,loc_x,o3,o3prod,o3loss,eminox,emivoc,tas,...,F_20,ch3o2_20,ch3ooh_20,loc_y,seq,circle,placename,land_earth,land_global,CONTINENT
1834,69.0,-49.0,1.0,68951.0,1.471148e-08,3.211922e-13,2.722755e-13,6.111121e-13,8.230395e-12,279.33667,...,-98907.228867,49451168.458313,5412797964.529861,11180,3081,69,French Southern & Antarctic Lands,1,1,Antarctica
2064,169.0,-45.0,1.0,168955.0,1.456359e-08,2.490233e-12,6.123487e-13,6.684854e-12,1.961126e-10,286.22003,...,-77901.899273,2815544.808921,31415830.044916,15682,7583,169,New Zealand,1,1,Oceania
2065,171.0,-45.0,1.0,170955.0,1.623399e-08,4.713648e-12,7.773185e-13,5.938116e-12,1.148444e-10,286.87515,...,-190024.81154,5586280.690937,66562224.452503,15772,7673,171,New Zealand,1,1,Oceania
2143,147.0,-43.0,1.0,146957.0,1.990147e-08,4.032379e-12,1.018624e-12,6.746558e-12,1.089857e-10,288.73615,...,-225148.212953,11244508.111218,232735942.379264,14693,6594,147,Australia,1,1,Australia
2155,171.0,-43.0,1.0,170957.0,1.529586e-08,2.395443e-12,7.764943e-13,5.167356e-12,2.154613e-10,288.01100,...,-74872.558739,4310641.701226,78051867.613989,15773,7674,171,New Zealand,1,1,Oceania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971712,-35.0,83.0,60.0,325083.0,1.944581e-08,1.879754e-16,3.207016e-16,8.447100e-14,4.913423e-16,235.54500,...,-18.572155,88516.087946,58774791.11931,6566,14667,325,Greenland,1,1,North America
971714,-31.0,83.0,60.0,329083.0,1.909269e-08,2.111646e-16,2.945706e-16,9.493963e-14,9.205029e-16,235.85202,...,-17.100392,45600.654215,26289061.710546,6746,14847,329,Greenland,1,1,North America
971715,-29.0,83.0,60.0,331083.0,1.874725e-08,2.106560e-16,1.801800e-16,1.413331e-13,1.779273e-15,235.96560,...,-10.927097,14744.870258,5984335.076487,6836,14937,331,Greenland,1,1,North America
971716,-27.0,83.0,60.0,333083.0,1.836742e-08,2.978477e-17,1.012632e-16,4.734165e-14,3.160282e-17,235.80554,...,-5.516713,117696.885483,84765946.028962,6926,15027,333,Greenland,1,1,North America


In [34]:
ho2_mean_land = dataset_land[['ho2_0','ho2_1','ho2_2','ho2_3','ho2_4','ho2_5',
                              'ho2_6','ho2_7','ho2_8','ho2_9','ho2_10',
                              'ho2_11','ho2_12','ho2_13','ho2_14','ho2_15',
                              'ho2_16','ho2_17','ho2_18','ho2_19','ho2_20']].mean()

In [35]:
ch3o2_mean_land = dataset_land[['ch3o2_0','ch3o2_1','ch3o2_2','ch3o2_3','ch3o2_4','ch3o2_5',
                                'ch3o2_6','ch3o2_7','ch3o2_8','ch3o2_9','ch3o2_10',
                                'ch3o2_11','ch3o2_12','ch3o2_13','ch3o2_14','ch3o2_15',
                                'ch3o2_16','ch3o2_17','ch3o2_18','ch3o2_19','ch3o2_20']].mean()

In [36]:
ch3ooh_mean_land = dataset_land[['ch3ooh_0','ch3ooh_1','ch3ooh_2','ch3ooh_3','ch3ooh_4','ch3ooh_5',
                                 'ch3ooh_6','ch3ooh_7','ch3ooh_8','ch3ooh_9','ch3ooh_10',
                                 'ch3ooh_11','ch3ooh_12','ch3ooh_13','ch3ooh_14','ch3ooh_15',
                                 'ch3ooh_16','ch3ooh_17','ch3ooh_18','ch3ooh_19','ch3ooh_20']].mean()

In [37]:
global_seq['ho2_mean_land'] = ho2_mean_land.values
global_seq['ch3o2_mean_land'] = ch3o2_mean_land.values
global_seq['ch3ooh_mean_land'] = ch3ooh_mean_land.values

In [38]:
global_seq

,iteration,ho2_mean_glob,ch3o2_mean_glob,ch3ooh_mean_glob,ho2_mean_land,ch3o2_mean_land,ch3ooh_mean_land
0,0,1.007553e+08,6.801647e+07,6.754535e+09,3.192089e+07,1.516037e+07,6.948275e+08
1,1,1.015822e+08,8.711602e+07,8.242158e+09,3.959130e+07,1.594287e+07,8.776082e+08
2,2,9.951722e+07,9.253380e+07,8.443686e+09,3.969327e+07,1.619936e+07,8.982037e+08
3,3,9.901203e+07,9.354321e+07,8.451055e+09,3.975410e+07,1.623499e+07,9.024641e+08
4,4,9.891048e+07,9.362798e+07,8.444239e+09,3.975849e+07,1.624294e+07,9.031771e+08
5,5,9.890837e+07,9.359856e+07,8.441956e+09,3.975966e+07,1.624437e+07,9.033187e+08
6,6,9.891491e+07,9.358202e+07,8.441723e+09,3.975982e+07,1.624466e+07,9.033454e+08
7,7,9.891792e+07,9.357772e+07,8.441846e+09,3.975985e+07,1.624472e+07,9.033508e+08
8,8,9.891869e+07,9.357729e+07,8.441924e+09,3.975986e+07,1.624473e+07,9.033518e+08
9,9,9.891878e+07,9.357748e+07,8.441949e+09,3.975986e+07,1.624473e+07,9.033521e+08


In [39]:
global_seq.to_csv('/Users/csuen/Documents/GitHub/ozone-radical/Extended Data Fig. 2/Extended Data Fig. 2 iteration convergence.csv', index=False)